In [1]:
import argparse
import os
import keras.backend as K
import keras.layers as KL
import numpy as np
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras import utils
from keras import Input, Model
from tqdm import tqdm

def prepare_data():
    """
    prepare mnist data
    """
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
    X_train = X_train.astype(np.float32) / 255.
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))
    X_test = X_test.astype(np.float32) / 255.

    y_train, y_test = utils.to_categorical(y_train, 10), utils.to_categorical(y_test, 10)

    return (X_train, y_train), (X_test, y_test)


class LeNet:
    """
    Before calling mc-dropout, use 'set_mc_dropout_rate' 
    """
    def __init__(self, input_shape, num_classes):
        self.mc_dropout_rate = K.variable(value=0)  # dropout before the last fully connected layer
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self._build_model()
        
    def _build_model(self):
        inp = Input(shape=self.input_shape)
        x = KL.Conv2D(filters=20, kernel_size=5, strides=1)(inp)
        x = KL.MaxPool2D(pool_size=2, strides=2)(x)
        x = KL.Conv2D(filters=50, kernel_size=5, strides=1)(x)
        x = KL.MaxPool2D(pool_size=2, strides=2)(x)
        x = KL.Flatten()(x)
        x = KL.Dense(500, activation='relu')(x)
        x = KL.Lambda(lambda x: K.dropout(x, level=self.mc_dropout_rate))(x)  # dropout before the last fully connected layer
        x = KL.Dense(self.num_classes, activation='softmax')(x)

        return Model(inputs=inp, outputs=x, name='lenet-mc-dropout')
    
    def set_mc_dropout_rate(self, new_rate):
        K.set_value(self.mc_dropout_rate, new_rate)
        
    def train(self, X_train, y_train, X_test, y_test,
              batch_size=32,
              epochs=2,
              optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['acc'],
              verbose=0):
        
        print(f"Training with mc_dropout_rate = {K.eval(self.mc_dropout_rate)}.\n")
        model = lenet.model
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

        # train the network
        model.fit(
            x=X_train,
            y=y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_test, y_test),
            verbose=verbose,
        )
        
(X_train, y_train), (X_test, y_test) = prepare_data()

Using TensorFlow backend.
C:\Users\Eli\Anaconda3\envs\tf1\lib\site-packages\google\protobuf\descriptor.py:47: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from google.protobuf.pyext import _message
C:\Users\Eli\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Eli\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\util\nest.py:823: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  _pywrap_tensorflow.RegisterType("Mapping", _collections.Mapping)
C:\Users\Eli\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\training\checkpointable\d

In [2]:
# prepare the model
lenet = LeNet(
    input_shape=X_train.shape[1:],
    num_classes=10,
)

# train
lenet.set_mc_dropout_rate(0.5)
lenet.train(X_train, y_train, X_test, y_test, epochs=20, verbose=2)

C:\Users\Eli\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\tensor_util.py:125: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  tensor_proto.float_val.extend([np.asscalar(x) for x in proto_values])


Instructions for updating:
Colocations handled automatically by placer.
Training with mc_dropout_rate = 0.5.

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
 - 21s - loss: 0.1600 - acc: 0.9510 - val_loss: 0.0667 - val_acc: 0.9794
Epoch 2/20
 - 20s - loss: 0.0724 - acc: 0.9773 - val_loss: 0.0610 - val_acc: 0.9833
Epoch 3/20
 - 20s - loss: 0.0552 - acc: 0.9830 - val_loss: 0.0521 - val_acc: 0.9832
Epoch 4/20
 - 20s - loss: 0.0484 - acc: 0.9855 - val_loss: 0.0652 - val_acc: 0.9821
Epoch 5/20
 - 20s - loss: 0.0402 - acc: 0.9876 - val_loss: 0.0500 - val_acc: 0.9863
Epoch 6/20
 - 20s - loss: 0.0393 - acc: 0.9882 - val_loss: 0.1023 - val_acc: 0.9749
Epoch 7/20
 - 20s - loss: 0.0376 - acc: 0.9892 - val_loss: 0.0712 - val_acc: 0.9814
Epoch 8/20
 - 20s - loss: 0.0348 - acc: 0.9897 - val_loss: 0.0714 - val_acc: 0.9826
Epoch 9/20
 - 20s - loss: 0.0333 - acc: 0.9905 - val_loss: 0.0702 - val_acc: 0.9844
Epoch 10/20
 - 21s - loss: 0.0326 - 

In [3]:
from gym_eli.envs import EliEnv
import gym

from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2

C:\Users\Eli\Anaconda3\envs\tf1\lib\site-packages\tensorflow\contrib\learn\python\learn\learn_io\generator_io.py:26: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Container
C:\Users\Eli\Anaconda3\envs\tf1\lib\site-packages\tensorflow\contrib\labeled_tensor\python\ops\core.py:724: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  tc.Union(string_types, tc.Tuple(string_types, collections.Hashable))),
C:\Users\Eli\Anaconda3\envs\tf1\lib\site-packages\tensorflow\contrib\lite\python\__init__.py:26: PendingDeprecationWarning: WARNING: TF Lite has moved from tf.contrib.lite to tf.lite. Please update your imports. This will be a breaking error in TensorFlow version 2.0.
  _warnings.warn(WARNING, PendingDeprecationWarning)


In [4]:
mask = 1000
X_train = X_train[:mask]
y_train = y_train[:mask]

# The algorithms require a vectorized environment to run
env = DummyVecEnv([lambda: EliEnv(net=lenet,
                                  confidence_rate=0.5, 
                                  X_train=X_train, 
                                  y_train=y_train,
                                  )])


in init


In [5]:
model = PPO2(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=500)


# obs = env.reset()
# for i in range(2000):

Instructions for updating:
Use keras.layers.flatten instead.


C:\Users\Eli\Anaconda3\envs\tf1\lib\site-packages\tensorflow\python\framework\tensor_util.py:131: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  tensor_proto.int_val.extend([np.asscalar(x) for x in proto_values])


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
in reset
in step
in take action
action=835


100%|█████████████████████████████████████████████████| 835/835 [01:13<00:00, 11.40it/s]


in reset
in step
in take action
action=838


...

in reset
in step
in take action
action=563


100%|█████████████████████████████████████████████████| 563/563 [00:49<00:00, 11.42it/s]


in reset
in step
in take action
action=54


100%|███████████████████████████████████████████████████| 54/54 [00:04<00:00, 11.42it/s]


in reset
-------------------------------------
| approxkl           | 0.0005241954 |
| clipfrac           | 0.0          |
| explained_variance | 3.27e-05     |
| fps                | 0            |
| n_updates          | 3            |
| policy_entropy     | 6.9049654    |
| policy_loss        | -0.0260822   |
| serial_timesteps   | 384          |
| time_elapsed       | 1.21e+04     |
| total_timesteps    | 384          |
| value_loss         | 494733.9     |
-------------------------------------
